# This file contain full description about how to plot figure 4.6 from Jafarkhani's book
---

## Prerequisites
---  
- `Python version = 3.10`
- Latest version of numpy
- Latest version of matplotlib

<b>Note:</b>  
All Functions used in this section can be found in `useful-function` folder.

:pen: <b>Author:</b> Amirhossein Solati  
:email: <b>Contact me:</b> aamirhosseinsolati@gmail.com

In [ ]:
import numpy as np
import os
# Import all our custom functions from the stbc_functions.py file
import functions.stbc_functions as stbc

In [3]:
SNRdB = np.arange(5, 30.1, 5)
SNR = 10**(SNRdB / 10) # Make dB format to linear similar to db2pow in matlab
Nb = int(1e5)  # Number of transmitted bits (for larger numbers -> take a while to have the answer)
varh_dB = 0 # For Rayleigh channel is 1
varh = 10**(varh_dB / 10) 
RESULTS_DIR = 'results'
RESULTS_FILE = os.path.join(RESULTS_DIR, 'simulation_results_4_6.npz') # Save Numerical Results in a file for reproducibility

In [4]:
# Create results directory if it doesn't exist
os.makedirs(RESULTS_DIR, exist_ok=True)

In [5]:
print("For N=1 and M=1 ---> SISO")
N, M = 1, 1 # Define Tx and Rx antennas as one cause it is SISO
Err1 = np.zeros(len(SNR)) # Give a empty vector of zeros which is gonna fill with counted errors
for i, snr_val in enumerate(SNR): # for all snr values we have to send Nb bit to find out BER at certain SNR
    Err = 0
    for _ in range(Nb):
        b1 = 2 * np.random.randint(0, 2) - 1 
        H = np.sqrt(varh / 2) * (np.random.randn(N, M) + 1j * np.random.randn(N, M))
        n0 = np.sqrt(1 / 2) * (np.random.randn(1, M) + 1j * np.random.randn(1, M))
        r0 = np.sqrt(snr_val / N) * b1 * H + n0
        ds_MLD = stbc.myMLD_BPSK_SISO(r0, H)
        if ds_MLD != b1:
            Err += 1
    Err1[i] = Err / Nb
    print(f"SNR (dB): {SNRdB[i]}, BER: {Err1[i]:.6f}")

For N=1 and M=1 ---> SISO
SNR (dB): 5.0, BER: 0.064790
SNR (dB): 10.0, BER: 0.022830
SNR (dB): 15.0, BER: 0.007870
SNR (dB): 20.0, BER: 0.002350
SNR (dB): 25.0, BER: 0.000740
SNR (dB): 30.0, BER: 0.000240


In [6]:
# For One iteration in above loop for SISO

print("\nGenerated random bit for bpsk:\n", b1)
print("\nGenerated random channel for bpsk:\n", H)
print("\nRecieved signal based in b1 and H:\n", r0)
print("\nDetect sent bit based on maximum likelihood:\n", ds_MLD)


Generated random bit for bpsk:
 -1

Generated random channel for bpsk:
 [[-0.81183449-0.63702094j]]

Recieved signal based in b1 and H:
 [[24.85646297+20.60539002j]]

Detect sent bit based on maximum likelihood:
 -1


In [7]:
print("For N=2 and M=1 ---> MISO")
N, M = 2, 1 # For 2 Tx and 1 Rx antennas
Err2 = np.zeros(len(SNR)) 
for i, snr_val in enumerate(SNR):
    Err = 0
    num_loops = int(Nb / N) # Transmit through 2 anetnnas
    for _ in range(num_loops):
        b1 = 2 * np.random.randint(0, 2, size=(N, 1)) - 1 # generate bit like (1,1)
        C = stbc.myAlamouti(b1) # pass generated bit to alamouti code to form C
        T = C.shape[0]  # TxN, so get first element of dimention (means row)
        H = np.sqrt(varh / 2) * (np.random.randn(N, M) + 1j * np.random.randn(N, M)) # Generate random channel
        n0 = np.sqrt(1 / 2) * (np.random.randn(T, M) + 1j * np.random.randn(T, M))  # generate random noise in TxM format
        r = np.sqrt(snr_val / N) * (C @ H) + n0 # received signal [ (TxN)*(NxM) -> (TxM) ]
        ds_MLD = stbc.myMLD_Alamouti_BPSK(r, H) # demodulate recived signal with maximum likelihood detector
        Err += np.sum(ds_MLD != b1) # count and sum up errors (first we detect the recieved bit, comapre with sent bit. If they are not equal, so we have error)
    Err2[i] = Err / (num_loops * N) # Count All Errors
    print(f"SNR (dB): {SNRdB[i]}, BER: {Err2[i]:.6f}")

For N=2 and M=1 ---> MISO
SNR (dB): 5.0, BER: 0.032690
SNR (dB): 10.0, BER: 0.005220
SNR (dB): 15.0, BER: 0.000540
SNR (dB): 20.0, BER: 0.000100
SNR (dB): 25.0, BER: 0.000000
SNR (dB): 30.0, BER: 0.000000


In [8]:
print("\nFor N = 3 and M=1 with bpsk modulation using code in (eq.4-61)")
N, M, K = 3, 1, 4 # K = number of symbols per block
Err3 = np.zeros(len(SNR))
for i, snr_val in enumerate(SNR):
    Err = 0
    num_loops = int(Nb / K) 
    for _ in range(num_loops):
        b1 = 2 * np.random.randint(0, 2, size=(K, 1)) - 1
        C = stbc.code_4_61(b1)
        T = C.shape[0]
        H = np.sqrt(varh / 2) * (np.random.randn(N, M) + 1j * np.random.randn(N, M))
        n0 = np.sqrt(1 / 2) * (np.random.randn(T, M) + 1j * np.random.randn(T, M))
        r = np.sqrt(snr_val / N) * (C @ H) + n0
        ds_MLD = stbc.myMLD_4_61_BPSK(r, H)
        Err += np.sum(ds_MLD != b1)
    Err3[i] = Err / (num_loops * K)
    print(f"SNR (dB): {SNRdB[i]}, BER: {Err3[i]:.6f}")

# All description above left as Alamouti coded signal, but here codeing scheme changed to equation 4.61    


For N = 3 and M=1 with bpsk modulation using code in (eq.4-61)
SNR (dB): 5.0, BER: 0.022540
SNR (dB): 10.0, BER: 0.002210
SNR (dB): 15.0, BER: 0.000130
SNR (dB): 20.0, BER: 0.000000
SNR (dB): 25.0, BER: 0.000000
SNR (dB): 30.0, BER: 0.000000


In [24]:
print("\n--- Sim 4: 4x1 BPSK (Code 4.38) ---")
N, M, K = 4, 1, 4
Err4 = np.zeros(len(SNR))
for i, snr_val in enumerate(SNR):
    Err = 0
    num_loops = int(Nb / K)
    for _ in range(num_loops):
        b1 = 2 * np.random.randint(0, 2, size=(K, 1)) - 1
        C = stbc.code_4_38(b1) # send code to
        T = C.shape[0]
        H = np.sqrt(varh / 2) * (np.random.randn(N, M) + 1j * np.random.randn(N, M))
        n0 = np.sqrt(1 / 2) * (np.random.randn(T, M) + 1j * np.random.randn(T, M))
        r = np.sqrt(snr_val / N) * (C @ H) + n0
        ds_MLD = stbc.myMLD_4_38_BPSK(r, H)
        Err += np.sum(ds_MLD != b1)
    Err4[i] = Err / (num_loops * K)
    print(f"SNR (dB): {SNRdB[i]}, BER: {Err4[i]:.6f}")


--- Sim 4: 4x1 BPSK (Code 4.38) ---
SNR (dB): 5.0, BER: 0.018560
SNR (dB): 10.0, BER: 0.000980
SNR (dB): 15.0, BER: 0.000010
SNR (dB): 20.0, BER: 0.000000
SNR (dB): 25.0, BER: 0.000000
SNR (dB): 30.0, BER: 0.000000


In [26]:
b1

array([[-1],
       [ 1],
       [ 1],
       [-1]])

In [21]:
print("\n--- Sim 5: 3x1 QPSK (Code G348) ---")
N, M, K_sym, bits_per_sym = 3, 1, 4, 2
bits_per_block = K_sym * bits_per_sym
Err5 = np.zeros(len(SNR))
for i, snr_val in enumerate(SNR):
    Err = 0
    num_loops = int(Nb / bits_per_block)
    for _ in range(num_loops):
        b0 = np.random.randint(0, 2, size=(bits_per_block, 1))
        b1 = stbc.myMod_QPSK(b0)
        C = stbc.code_G348(b1)
        T = C.shape[0]
        H = np.sqrt(varh / 2) * (np.random.randn(N, M) + 1j * np.random.randn(N, M))
        n0 = np.sqrt(1 / 2) * (np.random.randn(T, M) + 1j * np.random.randn(T, M))
        r = np.sqrt(snr_val / N) * (C @ H) + n0
        ds_MLD = stbc.myMLD_QPSK_G348(r, H)
        ds_MLD1 = stbc.myDemod_QPSK(ds_MLD)
        Err += np.sum(ds_MLD1 != b0)
    Err5[i] = Err / (num_loops * bits_per_block)
    print(f"SNR (dB): {SNRdB[i]}, BER: {Err5[i]:.6f}")


--- Sim 5: 3x1 QPSK (Code G348) ---
SNR (dB): 5.0, BER: 0.023580


KeyboardInterrupt: 

In [13]:
print("\n--- Sim 6: 4x1 QPSK (Code G448) ---")
N, M, K_sym, bits_per_sym = 4, 1, 4, 2
bits_per_block = K_sym * bits_per_sym
Err6 = np.zeros(len(SNR))
for i, snr_val in enumerate(SNR):
    Err = 0
    num_loops = int(Nb / bits_per_block)
    for _ in range(num_loops):
        b0 = np.random.randint(0, 2, size=(bits_per_block, 1))
        b1 = stbc.myMod_QPSK(b0)
        C = stbc.code_G448(b1)
        T = C.shape[0]
        H = np.sqrt(varh / 2) * (np.random.randn(N, M) + 1j * np.random.randn(N, M))
        n0 = np.sqrt(1 / 2) * (np.random.randn(T, M) + 1j * np.random.randn(T, M))
        r = np.sqrt(snr_val / N) * (C @ H) + n0
        ds_MLD = stbc.myMLD_QPSK_G448(r, H)
        ds_MLD1 = stbc.myDemod_QPSK(ds_MLD)
        Err += np.sum(ds_MLD1 != b0)
    Err6[i] = Err / (num_loops * bits_per_block)
    print(f"SNR (dB): {SNRdB[i]}, BER: {Err6[i]:.6f}")


--- Sim 6: 4x1 QPSK (Code G448) ---
SNR (dB): 5.0, BER: 0.018550
SNR (dB): 10.0, BER: 0.001040
SNR (dB): 15.0, BER: 0.000010
SNR (dB): 20.0, BER: 0.000000
SNR (dB): 25.0, BER: 0.000000
SNR (dB): 30.0, BER: 0.000000


In [14]:
np.savez(
    RESULTS_FILE,
    snr_db=SNRdB,
    err1=Err1, err2=Err2, err3=Err3,
    err4=Err4, err5=Err5, err6=Err6
)
print(f"\nSimulation complete. Results saved to '{RESULTS_FILE}'.")


Simulation complete. Results saved to 'results/simulation_results_4_6.npz'.


### Result for Figure 6 Chapter 4

<img src="plots/Figure_4_6.png" width=50%>